This third part is used for masking out unqualified data.\
neural case is defines as |Z/L|<0.1\
options: flux_magnitude_control, taylor_assumption_control, variance_control, angle_control, w_std_control

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os.path
import pickle
import time
from matplotlib.pyplot import figure
import scipy.io as sio
from datetime import date, timedelta
IN_DIRECTORY = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/basic_processed_data/"
IN_DIRECTORY2 = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/processed_data/"
OUT_DIRECTORY = "/projectnb/urbanclimate/yueqin/idaho_ec_jupyter/mask_data/"

in_var1=['H','u_star','L_H2','rot_ang_v']
in_var2=['u_std','u_avg','v_std','w_std','q_std','T_std']
out_var = ['mask_INL_all_1d','mask_neutral','mask_INL_all','mask_INL_all_11',
           'mask_INL_all_12','mask_INL_all_3','mask_INL_all_4','mask_INL_all_5']

# Defines functions

In [ ]:
def date_list(sdate,edate):
    """method used for creating date list"""
    delta = edate - sdate       # as timedelta
    day = [sdate+timedelta(days=x) for x in range(delta.days+1)]
    return day

# Define constants

In [ ]:
# set up controls
flux_magnitude_control    = 1    
taylor_assumption_control = 1    
variance_control          = 0    
angle_control             = 1 
w_std_control             = 0
do_saving = False

# some constants to be used
min_wnd                   = 120
max_wnd                   = 240 
sonum    =12                       # number of sonic
z        =[1.2,2,3.5,6,8.2,12.8,15.8,23,30.3,40.2,50.6,60.5]                 # height of sonic above ground, m
frequency=10                   # sampling frequency, Hz
time_avg =3600                  # average time, s
rpat = time_avg*frequency           # number of lines for a loop


# compute and save masks day by day

In [ ]:
%%time
Sdate = date(2020,9,25)
Edate = date(2021,4,23)
ds = date_list(Sdate,Edate)
t_id = 0
for day in ds:
    strday = str(day.strftime("%Y%m%d"))
    fp_stats = IN_DIRECTORY + 'u_ins_' + strday +'.pkl'
    if (not os.path.isfile(fp_stats)):
        print(day.strftime("%Y%m%d")+' do not exist')
        continue
    print('start loading:'+ strday)
    # load data
    for var in in_var1+in_var2: 
        a_file = open(IN_DIRECTORY + var +'_' + strday +'.pkl', "rb")
        globals()[var] = pickle.load(a_file)    
        a_file.close() 
    # calculate stability parameter
    z_o_L = np.zeros((len(L_H2),sonum))+np.nan
    for i in range(sonum):
        z_o_L[:,i]=z[i]/L_H2[:,i]
    ## create maskes
    mask_INL_all = np.zeros(np.shape(z_o_L))+1;   
    mask_INL_all_1d = np.zeros(np.shape(z_o_L)[0])+1;
    mask_INL_all_11 = np.zeros(np.shape(z_o_L))+1;  # H magnitude control
    mask_INL_all_12 = np.zeros(np.shape(z_o_L))+1;  # ustar magnitude control
    mask_INL_all_13 = np.zeros(np.shape(z_o_L))+1;  # LE magnitude control
    mask_INL_all_3 = np.zeros(np.shape(z_o_L))+1; # variance control
    mask_INL_all_4 = np.zeros(np.shape(z_o_L))+1; # angle control
    mask_INL_all_5 = np.zeros(np.shape(z_o_L))+1; # w_std control
    w_std_MOST = np.zeros(np.shape(z_o_L))+np.nan;
    # create mask for neutral condition
    mask_neutral = np.nanmax(abs(z_o_L),1)<0.1
    if flux_magnitude_control == 1:
        msk1 = abs(H)< 10
        mask_INL_all[msk1]= np.nan
        mask_INL_all_11[msk1]= np.nan
        msk2 = abs(u_star)<0.05
        mask_INL_all[msk2]= np.nan
        mask_INL_all_12[msk2]= np.nan
    if taylor_assumption_control == 1:
        msk3 = np.logical_or(u_std/u_avg>0.5,v_std/u_avg>0.5)
        mask_INL_all[msk3]= np.nan
        mask_INL_all_12[msk3]= np.nan
    if variance_control == 1:
        msk4 = np.logical_or(T_std<0.01 , u_std<0.005 , v_std<0.005 , \
        w_std<0.005 , q_std<2.35e-5)
        mask_INL_all[msk4] = np.nan
        mask_INL_all_3[msk4] = np.nan
    if angle_control == 1:
        msk5 = np.logical_and(rot_ang_v> min_wnd,rot_ang_v<max_wnd)
        mask_INL_all[msk5] = np.nan
        mask_INL_all_4[msk5] = np.nan
    if w_std_control == 1:
        msk = z_o_L < 0
        w_std_MOST[msk] = 1*(1-3*z_o_L[msk])^(1/3)
        msk6 = abs(w_std/u_star - w_std_MOST)> 0.2*w_std_MOST
        mask_INL_all[msk5] = np.nan
        mask_INL_all_5[msk5] = np.nan
    test=np.isnan(mask_INL_all)
    # 1-dimensional version of mask_INL_all
    mask_INL_all_1d[np.any(test,1)]= np.nan
    if do_saving:
        for var in out_var:
            a_file = open(OUT_DIRECTORY + var + '_' + strday +'.pkl', "wb")
            pickle.dump(eval(var), a_file)
            a_file.close()
    print('finish processing:'+ strday)  
    

In [ ]:
test=np.isnan(mask_INL_all)
mask_INL_all_1d

In [ ]:
pd.DataFrame(mask_INL_all)

In [ ]:
np.sum(test,1)>0

In [ ]:
np.sum(test,1)

In [ ]:
mask_INL_all_11

In [ ]:
mask_neutral